In [1]:
import pandas as pd
import numpy as np
# import trading_strategy_utils as tsu
import update_dataframe as ud
import grid_search as gs

In [18]:
# Load the BTC data
btc_df = pd.read_csv('data/BTC.csv', parse_dates=['ts_hour'])

# select the month from March to June (3 -6) in 2022
btc_df = btc_df[(btc_df['ts_hour'].dt.month >= 3) & (btc_df['ts_hour'].dt.month <= 4) & (btc_df['ts_hour'].dt.year == 2022)]
btc_df = btc_df[['ts_hour', 'pnl']]

btc_df.head()

,ts_hour,pnl
36369,2022-03-01 00:00:00,-0.002867
36370,2022-03-01 01:00:00,0.001325
36371,2022-03-01 02:00:00,0.002871
36372,2022-03-01 03:00:00,0.001481
36373,2022-03-01 04:00:00,0.001033


In [19]:
btc_df.tail()

,ts_hour,pnl
37825,2022-04-30 19:00:00,0.001176
37826,2022-04-30 20:00:00,0.000279
37827,2022-04-30 21:00:00,0.000063
37828,2022-04-30 22:00:00,0.000095
37829,2022-04-30 23:00:00,-0.000466


In [20]:
btc_df.shape

(1461, 2)

### Grid Search for EMA(Short & Long) params

In [21]:
# Define the range of parameters for grid search
short_window_range = range(4, 7, 1)
long_window_range = range(11, 15, 1)


# Perform grid search
grid_search= gs.grid_search_2_params(btc_df, short_window_range, long_window_range)
grid_search_sorted = grid_search.sort_values('total_weighted_pnl', ascending=False)

short_window: 4, long_window: 11, total_weighted_pnl: 0.06945709753356442
short_window: 4, long_window: 12, total_weighted_pnl: 0.07476457473960989
short_window: 4, long_window: 13, total_weighted_pnl: 0.07888660631990989
short_window: 4, long_window: 14, total_weighted_pnl: 0.0768792054854023
short_window: 5, long_window: 11, total_weighted_pnl: 0.08298379881398077
short_window: 5, long_window: 12, total_weighted_pnl: 0.07430578576137814
short_window: 5, long_window: 13, total_weighted_pnl: 0.075498004719279
short_window: 5, long_window: 14, total_weighted_pnl: 0.06846911760585442
short_window: 6, long_window: 11, total_weighted_pnl: 0.07657991060746365
short_window: 6, long_window: 12, total_weighted_pnl: 0.07305312606342018
short_window: 6, long_window: 13, total_weighted_pnl: 0.045548332846238604
short_window: 6, long_window: 14, total_weighted_pnl: 0.0444653797965454
Grid search finished.


In [22]:
grid_search_sorted

,short_window,long_window,total_weighted_pnl
4,5,11,0.082984
2,4,13,0.078887
3,4,14,0.076879
8,6,11,0.076580
6,5,13,0.075498
1,4,12,0.074765
5,5,12,0.074306
9,6,12,0.073053
0,4,11,0.069457
7,5,14,0.068469


# Run the trading strategy with the best EMA(Short & Long) params found by grid search

In [23]:
best_short_window = grid_search_sorted.iloc[0]['short_window']
best_long_window = grid_search_sorted.iloc[0]['long_window']
true_weight = 1

print('Best short window:', best_short_window)
print('Best long window:', best_long_window)
# Create a new DataFrame to store results with the best parameters
strategy_result = pd.DataFrame(columns=['ts_hour', 'pnl', 'EMA_short', 'EMA_long', 'is_positive', 'signal', 'count', 'consistency', 'is_continuous', 'class', 'weight_percentage', 'weight', 'weighted_pnl'])
results_list = []

# Update the DataFrame with the best parameters
for _, row in btc_df.iterrows():
    strategy_result = ud.update_dataframe_with_new_row(strategy_result, row, results_list, short_window=best_short_window, long_window=best_long_window, true_weight=true_weight)
print("Done")

Best short window: 5.0
Best long window: 11.0
Done


In [24]:
strategy_result[10:20].head(10)

,ts_hour,pnl,EMA_short,EMA_long,is_positive,signal,count,consistency,is_continuous,class,weight_percentage,weight,weighted_pnl
10,2022-03-01 10:00:00,0.000736,0.000665,0.000374,1,0,0,NaN,0,2.3,48.75,0.4875,0.0
11,2022-03-01 11:00:00,0.003181,0.001504,0.000842,1,0,0,NaN,0,2.3,48.75,0.4875,0.001551
12,2022-03-01 12:00:00,0.001617,0.001541,0.000971,1,0,0,NaN,0,2.3,48.75,0.4875,0.000788
13,2022-03-01 13:00:00,0.000354,0.001145,0.000868,1,0,0,NaN,0,2.3,48.75,0.4875,0.000172
14,2022-03-01 14:00:00,0.003344,0.001878,0.001281,1,0,0,NaN,0,2.3,48.75,0.4875,0.00163
15,2022-03-01 15:00:00,0.003955,0.002571,0.001727,1,0,0,NaN,0,2.3,48.75,0.4875,0.001928
16,2022-03-01 16:00:00,0.000027,0.001723,0.001443,1,0,0,NaN,0,2.3,48.75,0.4875,0.000013
17,2022-03-01 17:00:00,0.000153,0.001200,0.001228,1,-1,1,"[16, 17]",1,3.1,30,0.3,0.000075
18,2022-03-01 18:00:00,0.000428,0.000943,0.001095,1,0,-1,"[16, 17, 18]",1,2.2,48.75,0.4875,0.000129
19,2022-03-01 19:00:00,0.000414,0.000766,0.000981,1,0,0,NaN,0,2.3,48.75,0.4875,0.000202


In [25]:
strategy_result['class'].value_counts()

class
0      656
2.3    482
1      200
2.2    111
3.1     12
Name: count, dtype: int64

In [26]:
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
import nbformat

from plotly.subplots import make_subplots

In [27]:
# Create the cumulative weighted PnL column
strategy_result_copy = strategy_result.copy()
strategy_result_copy['cumulative_weighted_pnl'] = strategy_result_copy['weighted_pnl'].cumsum()

In [28]:
# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)


# Show the plot
fig.show()

In [29]:
# Define shape and color map for different classes
class_marker_map = {
    '1': {'color': 'red', 'symbol': 'circle', 'size': 10},
    '2.1': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.2': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.3': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '3.1': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.2': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.3': {'color': 'green', 'symbol': 'square', 'size': 6}
}

# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Add Class and Weight points to the plot
for class_type in strategy_result_copy['class'].unique():
    if class_type not in class_marker_map:
        continue
    class_df = strategy_result_copy[strategy_result_copy['class'] == class_type]
    marker = class_marker_map[str(class_type)]
    fig.add_trace(go.Scatter(
        x=class_df['ts_hour'],
        y=class_df['cumulative_weighted_pnl'],
        mode='markers',
        name=f'Class {class_type}',
        marker=dict(color=marker['color'], symbol=marker['symbol'], size=marker['size']),
        text=[f"Class: {row['class']}<br>Weight: {row['weight']:.2f}" for index, row in class_df.iterrows()],
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)

# Show the plot
fig.show()

In [30]:
strategy_result.weighted_pnl.sum()

0.08217633130075022